In [ ]:
# This R environment comes with many helpful analytics packages installed
# It is defined by the kaggle/rstats Docker image: https://github.com/kaggle/docker-rstats
# For example, here's a helpful package to load

library(tidyverse) # metapackage of all tidyverse packages

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

list.files(path = "../input")

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

library(sauron, lib='../input/sauron-package-explainable-cnn-troubleshooted')

In [ ]:
img_path<-file.path('../input/dataset-alzheimer/Alzheimer_s Dataset/test/ModerateDemented/27 (2).jpg')
library(magick);image_read(img_path)

In [ ]:
model<-load_model_hdf5('../input/alzheimer-reading-model/xception_dementia.h5')
summary(model)
size=300

In [ ]:
conv_base<-model %>% get_layer('xception')
final_dense<-model %>% get_layer('dense_2')
predict_layer<-conv_base$output %>% layer_flatten() %>% final_dense
custom_model<-keras_model(inputs=conv_base$input,
                         outputs=predict_layer)

In [ ]:
?image_load

In [ ]:
img_mat<-image_load(img_path, target_size=c(size,size)) %>%
    image_to_array() %>%
    array_reshape(dim=c(1,size,size,3))
img_mat<-img_mat/255
model %>% predict(img_mat)

In [ ]:
custom_rescale<-function(x){
    result<-x/255
    result
}

In [ ]:
?CNNexplainer

In [ ]:
explainer<-CNNexplainer$new(model=custom_model,
                       id='Xception_brain_MRI_reading',
                       preprocessing_function=custom_rescale)
explainer

In [ ]:
explainer$show_available_methods()
methods<-explainer$show_available_methods()$method

In [ ]:
explainations<-explainer$explain(input_imgs_paths=img_path,
                                methods=methods,
                                grayscale=F)
explainations$plot_and_save(combine_plots=F)